In [ ]:
from tools import analysistools as atools
from functools import partial as partial
from ga import networkedgeneticalgorithm as nga
from db import databaseconnection as dbconn

In [ ]:
epsIndices = range(4, 5)
epsDBs = ['/mnt/c/Users/manish/Documents/GitHub/optihedron/db/GP{}.db'.format(i) for i in epsIndices]
print(epsDBs[0])

In [ ]:
dropIndProps = partial(atools.dropChildren, parentKey = 'individuals', childKeys = ['genome', 'gh'])
dropGens = partial(atools.dropParentsConditional, parentKey = 'individuals', condition = lambda ind: ind['gen'] <= 30)
dropMetrics = partial(atools.dropParents, parentKey='metrics')
dropGenealogy = partial(atools.dropParents, parentKey='genealogy')

In [ ]:
dropTasks = [dropMetrics, dropGenealogy, dropIndProps, dropGens]
cleanTasks = [atools.cleanLigands, atools.cleanFits]
cleanTasks = [atools.cleanLigands]
sortTasks = [atools.sortIndGens]

epsData = atools.load(epsDBs, dbconn, clean = cleanTasks, drop = dropTasks, sort = sortTasks)

In [ ]:
epsLabel = 'ligand strength'
import seaborn as sns
import matplotlib.pyplot as plt
purpleMap = sns.cubehelix_palette(as_cmap=True)
BlueRedMap = 'RdBu_r'
plt.rcParams['figure.figsize'] = (10.0, 8.0)
atools.plotScanGen(epsData, epsLabel, epsIndices, atools.characterScore, epsIndices[0], 'MAX', 'characterscore-max', cmap=purpleMap, silent=False, backup=True)
atools.plotScanGen(epsData, epsLabel, epsIndices, atools.characterScore, epsIndices[0], 'MAX', 'characterscore-max2', cmap=BlueRedMap, annotate=True, vmin=-1, vmax=1)